In [5]:
%matplotlib inline

from __future__ import division
import numpy as np
import menpo.io as mio
from menpo.image import Image
from menpo.feature import greyscale, centralize, normalize_norm, normalize_std
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images
from alaborticcv2015.deepconvkernel.generative import GenerativeLDCN, learn_pca_filters

# Generate PCA Net:

Choose whether to work with color or greyscale images and whether to learn the filters from landmarks or regular grid:

In [6]:
folder_path = '/data/PhD/alaborticcv2015/'

In [7]:
color = False
landmarks = True

if color:
    n_channels = 3
else:
    n_channels = 1

In [8]:
images = []
for i in mio.import_images('/data/PhD/DataBases/faces/lfpw/**/*.png', verbose=True, 
                           max_images=1000):
    i.crop_to_landmarks_proportion_inplace(0.5)
    i = i.rescale_landmarks_to_diagonal_range(100)
    i = greyscale(i)
    labeller(i, 'PTS', ibug_face_66)
    images.append(i)

- Loading 1000 assets: [====================] 100%

/data/pythondev/src/menpo/menpo/image/base.py:130: UserWarning: The copy flag was NOT honoured. A copy HAS been made. Please ensure the data you pass is C-contiguous.
  warn('The copy flag was NOT honoured. A copy HAS been made. '


Build PCA Net 1 layer:

In [13]:
n_filters = range(61, 81, 1)
patch_shape = [ps for ps in range(2, 11, 1)]

In [14]:
for nf in n_filters:
    for ps in patch_shape:
        
        if nf < ps * ps:

            net = GenerativeLDCN(learn_filters=learn_pca_filters, 
                                 n_filters=np.minimum(nf, ps * ps -1),
                                 n_layers=1,
                                 architecture=3,
                                 normalize_patches=centralize,
                                 normalize_filters=None,
                                 patch_shape=(ps, ps))
            if landmarks:
                net.learn_network_from_landmarks(images, group='ibug_face_66')
            else:
                net.learn_network_from_grid(images, stride=(4, 4))

            file_name = (folder_path
                         + 'networks/pca_net/pca_net_1_1_' 
                         + str(nf) 
                         + '_' 
                         + str(ps)
                         + '.pkl.gz') 
            mio.export_pickle(net, file_name, overwrite=True)

            print '- network {}, {}: '.format(nf, ps)

- network 61, 8: 
- network 61, 9: 
- network 61, 10: 
- network 62, 8: 
- network 62, 9: 
- network 62, 10: 
- network 63, 8: 
- network 63, 9: 
- network 63, 10: 
- network 64, 9: 
- network 64, 10: 
- network 65, 9: 
- network 65, 10: 
- network 66, 9: 
- network 66, 10: 
- network 67, 9: 
- network 67, 10: 
- network 68, 9: 
- network 68, 10: 
- network 69, 9: 
- network 69, 10: 
- network 70, 9: 
- network 70, 10: 
- network 71, 9: 
- network 71, 10: 
- network 72, 9: 
- network 72, 10: 
- network 73, 9: 
- network 73, 10: 
- network 74, 9: 
- network 74, 10: 
- network 75, 9: 
- network 75, 10: 
- network 76, 9: 
- network 76, 10: 
- network 77, 9: 
- network 77, 10: 
- network 78, 9: 
- network 78, 10: 
- network 79, 9: 
- network 79, 10: 
- network 80, 9: 
- network 80, 10: 


Build PCA Net 2 layer:

In [17]:
for nf in n_filters:
    for ps in [5]:#patch_shape:

        nf = 8 #ps * ps -1

        net = GenerativeLDCN(learn_filters=learn_pca_filters, 
                             n_filters=np.minimum(nf, ps * ps -1),
                             n_layers=2,
                             architecture=1,
                             normalize_patches=centralize,
                             normalize_filters=None,
                             patch_shape=(ps, ps))
        if landmarks:
            net.learn_network_from_landmarks(images, group='ibug_face_66')
        else:
            net.learn_network_from_grid(images, stride=(4, 4))

        file_name = (folder_path
                     + 'networks/pca_net/pca_net_2_1_' 
                     + str(nf) 
                     + '_' 
                     + str(ps)
                     + '.pkl.gz') 
        mio.export_pickle(net, file_name, overwrite=True)

        print '- network {}, {}: '.format(nf, ps)

- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 
- network 8, 5: 


In [ ]:
net.kernels_frequency((50,50))[-1].view()